# Preprocessing

### Libraries

In [2]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import os

### Load dataset (user acess)

In [ ]:


# Directory containing the CSV files
input_folder = 'C:/Users/eng3/Documents/pos/Tech Challenger 5/dataset/files/treino'

dataframes = []

for file_name in os.listdir(input_folder):
    if file_name.endswith('.csv'):  
        file_path = os.path.join(input_folder, file_name)
        df = pd.read_csv(file_path)
        dataframes.append(df)

combined_df = pd.concat(dataframes, ignore_index=True)

df_user_acc = combined_df

required_columns = ['history', 'timestampHistory', 'numberOfClicksHistory', 'timeOnPageHistory', 
                    'scrollPercentageHistory', 'pageVisitsCountHistory']
df_user_acc = df_user_acc.dropna(subset=required_columns)

df_user_acc.head()


### Load dataset (news metadata)

In [ ]:


# Directory containing the CSV files
input_folder = 'C:/Users/eng3/Documents/pos/Tech Challenger 5/dataset/itens/itens/filtered_output'

dataframes = []

for file_name in os.listdir(input_folder):
    if file_name.endswith('.csv'):  
        file_path = os.path.join(input_folder, file_name)
        df = pd.read_csv(file_path)
        dataframes.append(df)

combined_df = pd.concat(dataframes, ignore_index=True)

df_news_acc = combined_df

df_news_acc.head()


In [ ]:
# Step 1: Preprocess df_news_acc to create a mapping
page_to_article_type = df_news_acc.set_index('page')['article-type'].to_dict()

# Step 2: Define a faster function using the preprocessed dictionary
def process_values(row):
    # Remove spaces and split history into IDs
    news_ids = row.replace(" ", "").split(',')
    
    # Map IDs to their article types using the dictionary
    ret = [page_to_article_type.get(news, "Unknown") for news in news_ids]
    
    # Join the result as a comma-separated string
    retString = ""

    try:
        retString = ",".join(ret)
    except:
        retString = ""
        
    return retString


# Step 3: Apply the optimized function to the 'history' column
df_user_acc['article_types'] = df_user_acc['history'].apply(process_values)

# Step 4 (optional): Check the results
print(df_user_acc.head())


In [ ]:
df_user_acc['history'] = df_user_acc['history'].astype(str).apply(lambda x: x.split(', '))
df_user_acc['article_types'] = df_user_acc['article_types'].astype(str).apply(lambda x: x.split(',') if x != 'nan' else [])

# Export to parquet
df_user_acc.to_parquet('recommendation.parquet', index=False)

print("DataFrame exported to 'recommendation.parquet'")

In [ ]:
df_news_acc.to_parquet('news_processed.parquet')

print("DataFrame exported to 'news_processed.parquet'")